## Load ground truth

In [1]:
import pandas as pd
import json
topics_file_path = 'output/llm_output/topics.json'
with open(topics_file_path, 'r') as f:
    topics = json.load(f)
ground_truth = [dict(topic_name = x['topic_name'], 
                     ts_msg_id = x['msg_ids'][0],
                     question = x['question'],
                     answer = x['answer'],
                     ) for x in topics]
ground_truth[0]




{'topic_name': 'Медицинский чекап и очереди',
 'ts_msg_id': 186659,
 'question': 'What is the best time to go for a medical checkup to avoid long queues?',
 'answer': 'Going after lunch might be advisable as it seems there are generally fewer people, and the process is quicker.'}

### gettind data from summarization by topics, where we already have question and answer, that can be used as ground truth data

In [4]:
topics_file_path = 'output/llm_output/topics.json'
topics_file_path_gt = 'output/llm_output/ground_truth.json'

In [ ]:
from tqdm.auto import tqdm
from src.rag_integration import RaguDuDu
rg = RaguDuDu()
with open(topics_file_path, 'r') as f:
    topics = json.load(f)
ground_truth = [dict(topic_name=x['topic_name'],
                        ts_msg_id=x['msg_ids'][0],
                        question=x['question'],
                        answer=x['answer'],
                    ) for x in topics]
for topic in tqdm(ground_truth):
    question = topic['question']
    rag_answer = rg.rag_by_dense_vector_search(question=question)
    topic["rag_answer"] = rag_answer
    with open(topics_file_path_gt, 'w') as fl:
        json.dump(ground_truth, fl, ensure_ascii=False, indent=4)


## Cosine similarity


### gpt-4o

In [6]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

/Users/dklmn/.local/share/virtualenvs/telegram_rag_search-oI0nq4RM/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
def compute_similarity(record):
    answer_orig = record['answer']
    answer_llm = record['rag_answer']
    
    v_llm = model.encode(answer_llm)
    v_orig = model.encode(answer_orig)
    
    return v_llm.dot(v_orig)

In [10]:
with open(topics_file_path_gt, 'r') as f:
    dd = json.load(f)
similarity = []
for record in tqdm(dd):
    sim = compute_similarity(record)
    record['cs'] = sim
    similarity.append(sim)

100%|██████████| 51/51 [00:02<00:00, 19.24it/s]


In [11]:
import pandas as pd
df = pd.DataFrame(dd)
df

,topic_name,ts_msg_id,question,answer,rag_answer,cs
0,Медицинский чекап и очереди,186659,What is the best time to go for a medical chec...,Going after lunch might be advisable as it see...,"To avoid long queues for a medical checkup, it...",0.461599
1,Продление ВНЖ и роль страхования SGK,188347,Is SGK insurance mandatory for the renewal of ...,SGK is not mandatory for the renewal of a resi...,SGK insurance is not mandatory for the renewal...,0.949372
2,Проблемы транспорта и работа в офисе,192255,What are the challenges of commuting to the of...,The challenges include high vehicle costs and ...,Commuting to the office from Antalya poses sev...,0.433188
3,Прописка иностранцев в арендуемом жилье,183632,Is it possible to register a non-relative fore...,It is generally difficult to register non-rela...,It seems registering a non-relative foreigner ...,0.841946
4,Бюрократия и получение справки о несудимости,183784,What can be done when strange requirements com...,Participants suggest asking to see the law on ...,Obtaining a criminal record certificate might ...,0.217937
5,Романтические поездки и фото в Каппадокии,184130,What is the best time for a romantic photo ses...,The best time for a romantic photo session in ...,The best time for a romantic photo session in ...,0.934400
6,Обсуждение переездов и жизни в Турции,185322,What are the challenges people face when reloc...,"Challenges include rising taxes, high costs, l...",Challenges people face when relocating to Turk...,0.831371
7,Поиски квартир в Анталии,186146,Where might available apartments be listed out...,Apartments might be listed with individual age...,Apartments might be available on Facebook grou...,0.572070
8,Такси и маршруты в Анталии,188528,How does the taxi fare calculation in Antalya ...,The taxi fare in Antalya is calculated based o...,Taxi fare calculation in Antalya depends on bo...,0.853220
9,"цены, блокировки и перспективы переезда",185092,What are the current issues people face in Tur...,Current issues in Turkey include rising prices...,The current issues discussed by people in Turk...,0.850798


In [12]:

df['cs'].describe()

count    51.000000
mean      0.608614
std       0.313047
min      -0.085391
25%       0.447394
50%       0.739580
75%       0.832274
max       0.951847
Name: cs, dtype: float64